## Initializing Elasitcsearch

We begin by initializing Elasticsearch with the names of our index and type. You should have Elasticsearch installed and already run the "bin/elasticsearch" command. For more information you can visit: https://www.elastic.co/guide/en/elasticsearch/guide/current/running-elasticsearch.html.

Here we initialize an object where corpus will be the name of the index with a single type articles, and a field named sentence. It will automatically index the Document from our snorkelSession. 

By default each document will contain a field corresponding to the sentence ID number, the sentence and an empty vector of 'o's (this will be useful for generating candidate tags below). Once everything has been indexed we can see its status with the amount of documents it contains as well as the size.

### Generate Tags

Using our Spouse candidates we will generate a new field that tags the corresponding position in the sentence. This is done using the `cands` keyword when initializing the session, or with the `generateTags` function.

In [2]:
from snorkel import SnorkelSession
from snorkel.models import candidate_subclass
from elastics import elasticSession ,printResults,deleteIndex

# import os

# # TO USE A DATABASE OTHER THAN SQLITE, USE THIS LINE
# # Note that this is necessary for parallel execution amongst other things...
# os.environ['SNORKELDB'] = 'postgres:///snorkel-intro'
# deleteIndex("_all")
#Repeat the Spouse definition from the tutorial
session = SnorkelSession()
Spouse = candidate_subclass('Spouse', ['person1', 'person2'])
print "asdf"
for c in session.query(Spouse).all():
    print c
    print "hi"
    break
# eSearch=elasticSession(cands=Spouse)

#To delete the corpus index, needed to re-run
# deleteIndex("corpus")

asdf


## Querying the Corpus

The type of query that we perform is defined by the first arguement. Each query also contains an optional size and slop keyword parameter. Size specifies the number of results we wish to return and slop is the amount of acceptable positions the values in the query can be away from each other. 

Size and slop as defaulted to 5 and 0 respectively.
https://www.elastic.co/guide/en/elasticsearch/guide/current/slop.html

To display the results we can use the printResult function, which takes the search result as its first parameter and the names of the fields you wish to print out.

*Queries are case sensitive


### Arbitrary Search

Once we have all our documents indexed we can perform a simple query. Using the `match` keyword, we query our sentence field in every document for the words married OR children. Matches that contain both and their entirety will be scored higher. After performing the query we print the results which are sorted in a decsending order. 

https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-match-query.html

In [ ]:
query="married children"
searchResult = eSearch.searchIndex("match",query)
printResults(searchResult,"sentence")

### Search contain all values in query

Specifying a slop parameter will force the query to only return results that contain every word in the query. Explicitly stating a slop=0 will return results where the query values appear side by side. This query looks for white AND trousers that occur in a sentence. 

https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-match-query-phrase.html

In [ ]:
query="white trousers"
searchResult = eSearch.searchIndex("match",query,slop=0,size=3)
printResults(searchResult,"sentence")

### Positional Search

Using a slop value for the above queries return a result that longer enforces the ordering of our query.  To search for a query where we want care about the ordering we would use the `position` keyword. This time specifying the slop defines the number of acceptable position shifts to have the query side by side, respective of order. This query is different from the `match` search above as this enforces an ordering of the query terms.

The query string has to be passed in as a parameter as there individual terms. This type of query accepts any number of terms.

In [ ]:
searchResult = eSearch.searchIndex("position","Pennsylvania","Colorado",slop =20)
printResults(searchResult,"sentence")

### Search between Candidates

We can also search in between two candidates which were defined as PERSON in the spousal tutorial. Specifically, we are querying for PERSON married PERSON in that order. To do this we use the `inCand` keyword,followed by the word we want to search for.

*All candidate searches only allow for a singular term

In [ ]:
result = eSearch.searchIndex("inCand","married",slop=100)
printResults(result,"sentence","tagged","lineNum")


### Search before candidate 

Using the `bCand` keyword we can search for the occurance of a term that appears before any PERSON. 

In [ ]:
result = eSearch.searchIndex("bCand","married",slop=100,size=3)
printResults(result,"sentence")

### Search after candidate 

Using the `aCand` keyword we can search for the occurance of a term that appears after any PERSON. 

In [ ]:
result = eSearch.searchIndex("aCand","married",slop=100,size=3)
printResults(result,"sentence")

### Visualizing a Document

We can get any document by it's line number. Here get the first sentence in our corpus and print the values stored in each field

In [ ]:
result = eSearch.getDoc(42743)
print "Sentence"
print result['_source']['sentence']
print "Sentence number"
print result['_source']['lineNum']
print "Tagged Candidates"
print result['_source']['tagged']